# Wikipédia - Lista de municípios do Brasil por IDH
        Mais um exercício de prática de web scraping.
        Vamos pegar os dados de uma tabela da wikipédia.

In [332]:
# Seção imports

import requests
from bs4 import BeautifulSoup

# Para criar o dataset
import pandas as pd

In [2]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_IDH'

####  Pegando o documento html

In [25]:
html = requests.get(url).text

#### BeautifulSoup é um objeto que nos permite trabalhar com o html recebido

In [26]:
soup = BeautifulSoup(html)

#### Visualizando a 'sopa'. .prettify() torna o objeto mais legível   (teste sem utilizar prettify e admire)

In [29]:
print(soup.prettify()[0:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Lista de municípios do Brasil por IDH – Wikipédia, a enciclopédia livre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","nove


#### Obs: Podemos usar atributos(tagsdo html) para encontrar as informações que queremos.        

In [30]:
soup.title

<title>Lista de municípios do Brasil por IDH – Wikipédia, a enciclopédia livre</title>

In [31]:
# Para obter apenas a informação (sem as <tags>) .text é uma opção

soup.title.text

'Lista de municípios do Brasil por IDH – Wikipédia, a enciclopédia livre'

### Direto para a ação

In [215]:
table = soup.find('table', class_= "wikitable sortable")

In [230]:
'''
    Algo que notei, os dados que queremos têm as mesmas tags (e nenhuma classificação),
    ou seja, ao tentar pegar apenas as cidades, o índice e UF vêm juntos \o\|o|/o/
    Encontrei uma forma um tanto quanto enrolada para pegar cada um separadamente.
'''

table.td

<td>1
</td>

In [228]:
table.td.nextSibling

'\n'

In [229]:
table.td.nextSibling.nextSibling

<td><a href="/wiki/S%C3%A3o_Caetano_do_Sul" title="São Caetano do Sul">São Caetano do Sul</a>
</td>

#### Pegando os dados

In [290]:
all_data = soup.find('table', class_= "wikitable sortable")

ponto_de_parada = 0 # Para não mostrar a lista tooooda

for city in all_data.findAll('td'):
    print(city.text)
    print('\__________________/--->',ponto_de_parada,'')
    
    ponto_de_parada+=1
    if ponto_de_parada == 8: break

1

\__________________/---> 0 
São Caetano do Sul

\__________________/---> 1 
 São Paulo

\__________________/---> 2 
0,862

\__________________/---> 3 
2

\__________________/---> 4 
Águas de São Pedro

\__________________/---> 5 
 São Paulo

\__________________/---> 6 
0,854

\__________________/---> 7 


#### Passando dados para uma lista

In [289]:
all_data = soup.find('table', class_= "wikitable sortable")

all_in_list = []

for city in all_data.findAll('td'):
    all_in_list.append(city.text)
    
all_in_list[-8:-1]

['100\n',
 'Ivoti\n',
 '\xa0Rio Grande do Sul\n',
 '0,784\n',
 '100\n',
 'Santa Maria\n',
 '\xa0Rio Grande do Sul\n']

#### Utilizando os 'pulos' de 4 em 4 para mapear as cidades, as UF e os IDHs

In [247]:
# Sabendo quantos índices temos ao total (para saber até onde 'pular')
number_of_itens_in_list = 0

for i in all_in_list:
    number_of_itens_in_list += 1

number_of_itens_in_list

436

In [257]:
# Capturando as cidades e colocando-as em uma lista

index=1 
cities_list = []

while index < number_of_itens_in_list: 
    cities_list.append(all_in_list[index])
    index += 4

cities_list[-6:] # os últimos itens batem com os últimos da tabela do site yyeeeyy \(*-*)/

['Ribeirão Pires\n',
 'Santa Fé do Sul\n',
 'Salto Veloso\n',
 'Timbó\n',
 'Ivoti\n',
 'Santa Maria\n']

#### Agora fazemos o mesmo para a UF e IDH. Lembrando de aumentar o valor do índice/index.

In [260]:
index=2 
uf_list = []

while index < number_of_itens_in_list: 
    uf_list.append(all_in_list[index])
    index += 4

uf_list[-6:] 

['\xa0São Paulo\n',
 '\xa0São Paulo\n',
 '\xa0Santa Catarina\n',
 '\xa0Santa Catarina\n',
 '\xa0Rio Grande do Sul\n',
 '\xa0Rio Grande do Sul\n']

In [265]:
index=3 
idh_list = []

while index < number_of_itens_in_list: 
    idh_list.append(all_in_list[index])
    index += 4

idh_list[-2:]

['0,784\n', '0,784\n']

In [267]:
# Confirmando que as listas têm o mesmo número de itens (logo, nenhum ficou de fora ou veio como intruso)

print(len(cities_list), len(uf_list), len(idh_list))

109 109 109


In [272]:
# Comparando no site. Sim, os dados estão corretos :)

print(cities_list[0], uf_list[0], idh_list[0]) 
print('----------')
print(cities_list[108], uf_list[108], idh_list[108]) 

São Caetano do Sul
  São Paulo
 0,862

----------
Santa Maria
  Rio Grande do Sul
 0,784



#### Agora só precisamos pegar os nomes das colunas

In [321]:
looking_for_headers =  soup.find('table', class_= "wikitable sortable")

# Quase lá, mais ou menos 
looking_for_headers.text[0:100]

'\n\n#\n\nMunicípio\n\nUF\n\nIDHM (2010)[5]\n\n\n1\n\nSão Caetano do Sul\n\n\xa0São Paulo\n\n0,862\n\n\n2\n\nÁguas de São Pedr'

In [322]:
# Encontramos
looking_for_headers.findAll("a")[0:3]

[<a href="/wiki/Munic%C3%ADpio" title="Município">Município</a>,
 <a class="mw-redirect" href="/wiki/Unidades_federativas_do_Brasil" title="Unidades federativas do Brasil">UF</a>,
 <a class="mw-redirect" href="/wiki/IDH" title="IDH">IDHM</a>]

In [327]:
headers = looking_for_headers.findAll("a")[0:3]

headers[0]

<a href="/wiki/Munic%C3%ADpio" title="Município">Município</a>

In [328]:
headers[0].text

'Município'

In [330]:
# No site está escrito IDHM mesmo
for header in headers:
    print(header.text)

Município
UF
IDHM


In [331]:
# Pronto para passarmos para uma lista
header_list = []

for header in headers:
    header_list.append(header.text)
    
header_list

['Município', 'UF', 'IDHM']

#### Criando o DataFrame e passando-o para um csv

In [376]:
# Criando um
data_frame = pd.DataFrame(cities_list, columns=['Cidade'])

# Criando outro e já o adicionando ao primeiro
data_frame = data_frame.join( pd.DataFrame(uf_list, columns=['UF']) )

# Hehe, não diria que é a melhor alternativa, mas por hora farei assim
data_frame = data_frame.join( pd.DataFrame(idh_list, columns=['IDH']) )

data_frame

# Como pode ver, ainda há '\n' símbolos, mas podemos solucionar isso durante a análise exploratória (EDA)

,Cidade,UF,IDH
0,São Caetano do Sul\n,São Paulo\n,"0,862\n"
1,Águas de São Pedro\n,São Paulo\n,"0,854\n"
2,Florianópolis\n,Santa Catarina\n,"0,847\n"
3,Balneário Camboriú\n,Santa Catarina\n,"0,845\n"
4,Vitória\n,Espírito Santo\n,"0,845\n"
...,...,...,...
104,Santa Fé do Sul\n,São Paulo\n,"0,784\n"
105,Salto Veloso\n,Santa Catarina\n,"0,784\n"
106,Timbó\n,Santa Catarina\n,"0,784\n"
107,Ivoti\n,Rio Grande do Sul\n,"0,784\n"


In [377]:
data_frame.to_csv('100 maiores IDHs brasileiros.csv')